You have a number of envelopes with widths and heights given as a pair of integers `(w, h)`. One envelope can fit into another if and only if both the width and height of one envelope is greater than the width and height of the other envelope.

What is the maximum number of envelopes can you Russian doll? (put one inside other)

Note:
Rotation is not allowed.

So right off the bat I think I should sort the list. Though perhaps it wouldn't matter. In theory, if we had a table of which envelopes fit inside other envelopes, then we could just recursively iterate and find the answer.

After beginning to think about the problem, I decided the direction to proceed is to let DP(i) be the number of maximum number of envelopes involving the ith envelope. Then DP(j) is the maximum over all DP(i) where i < j and `envelopes[j]` contains `envelopes[i]` and 1 (the case where the jth envelope doesn't contain any earlier envelopes).

In [29]:
class Solution:
    def maxEnvelopes(self, envelopes) -> int:
        n = len(envelopes)
        envelopes.sort() # O(n*log(n)), n is the length of envelopes
        
        D = []
        for i, (a, b) in enumerate(envelopes):
            max_envelope = 1
            for j in range(i):
                c, d = envelopes[j]
                if c < a and d < b:
                    max_envelope = max(max_envelope, D[j]+1)
            D.append(max_envelope)
        # O(n**2)
        
        # We handle the case when n == 0
        D.append(0)
        
        return max(D)

9168 ms (5.08%) and 15.1 MB (20.00%)  
Runtime Complexity: O(n\*\*2) where `n` is the number of envelopes.  
Extra Space Complexity: O(n)

One of the main problems of the above is that it makes some unnecessary envelope containment checks. For example if `envelopes[j]` contains `envelopes[i]` and `envelopes[k]` contains `envelopes[j]` then we don't need to check the relationship between `envelopes[i]` and `envelopes[k]`.

What we can do instead is create a directed graph and only check containment of a node's children if the parent node is not contained.

In [45]:
class Solution:
    def maxEnvelopes(self, envelopes) -> int:
        n = len(envelopes)
        envelopes.sort()
        
        parents = []
        node_children = {}
        
        def is_one_inside_two(envelope1, envelope2, visited):
            if envelope1[0] < envelope2[0] and envelope1[1] < envelope2[1]:
                node_children.setdefault(envelope2, {}).setdefault("children", []).append(envelope1)
                return (node_children[envelope1]["max"], True)
            else:
                res = 0
                for child in node_children[envelope1]:
                    if child not in visited:
                        visited.add(child)
                        res = max(res, is_one_inside_two(child, envelope2, visited)[0])
                return (res, False)
                    
        for i, (a, b) in enumerate(envelopes):
            max_envelope = 1
            new_parents = []
            for parent in list(parents):
                res, is_contained = is_one_inside_two(parent, (a, b), set())
                max_envelope = max(res, max_envelope)
                if not is_contained:
                    new_parents.append(parent)
            new_parents.append((a, b))
            parents = new_parents
            node_children.setdefault((a, b), {}).setdefault("max", max_envelope)
        
        return max([node_children[parent]["max"] for parent in parents])


Worst-case: O(n\*\*2)
Best-case: O(n)

In [46]:
class SolutionTester(Solution):
    def test(self):
        testcase1 = ([[5,4],[6,4],[6,7],[2,3]], 3)     # Example on LeetCode
        
        testcase2 = ([], 0)
        testcase3 = ([[1,2]], 1)
        testcase4 = ([[1,2],[2,1]], 1)
        testcase5 = ([[1,2],[1,3]], 1)
        testcase6 = ([[2,3],[3,3]], 1)
        testcase7 = ([[1,2],[2,3],[2,4],[3,2],[3,3],[3,4]], 3)
        testcase8 = ([[1,2],[2,3],[2,4],[3,4],[4,5],[5,6]], 5)
        testcase9 = ([[10,10]], 1)
        
        testcases = [testcase1, testcase2, testcase3,
                     testcase4, testcase5, testcase6, 
                     testcase7, testcase8, testcase9]
        
        for test_input, expected_output in testcases:
            print("Input:", test_input)
            sol_output = self.maxEnvelopes(test_input)
            print("Output:", sol_output)
            print("Expected Output:", expected_output)
            assert sol_output == expected_output, "Testcase failed!"
            print()

In [47]:
s = SolutionTester()

In [48]:
s.test()

Input: [[5, 4], [6, 4], [6, 7], [2, 3]]


TypeError: '<' not supported between instances of 'str' and 'int'